# Lecture 14 - Nonlinear optimization

So far we have looked at methods to solve single equations of a single variable, such as
$$x=f(x)\ \ \textrm{ and } f(x)=0$$

We need to now extend these methods to solve equations involving multiple variables, such as
$$x=f(x,y), \ y=g(x,y),$$
or
$$\begin{array}{c}
x_1 &=& f_1(x_1,\dots,x_N), \\
 & \vdots & \\
x_N &=& f_N(x_1,\dots,x_N)
\end{array}$$
or 
$$\begin{array}{c}
f_1(x_1,\dots,x_N) &=& 0, \\
 \vdots & \\
f_N(x_1,\dots,x_N) &=& 0
\end{array}$$

In [1]:
%matplotlib inline
import numpy
import scipy.misc
import scipy.optimize
import matplotlib.pyplot as plot

#### Relaxation method

The relaxation method can be extended easily to multiple variables. Just as we saw with one variable, it is important to try different ways of writing the equations to see how they converge to a solution.

#### Example 1
Determine the intersection point between a line and a parabola. The equations are $$ y=ax+b,\ y=x^2+c.$$ There are two ways of solving one equation for $x$ and the other for $y$.
$$x'=\frac{y-b}{a}, \ y' = x^2+c$$
or
$$x' = \sqrt{y-c}, \ y'= ax+b$$

In [11]:
# -- I am using our new data container; the tuple.
# -- Tuples are unchangable lists that are used for 
# -- moving collection of data from place to place in your code.
# --
# -- Tuples use (,). Lists use [,]
# --
# -- With these values of a,b,c there should be two intersections
# -- at (1,2) and (0,1)
constants = ( 1.0, # a
              1.0, # b
              1.0) # c
# x = 0.9 # USE f0 to find intersection at (0,1)
# y = 0.1
x = 1.9  # USE f1  to find intersection at (1,2)
y = 2.1

# -- See how the varible args will contain the 
# -- tuple 'constants', and is unpacked to reveal a,b,c
# -- This way I need only to change a,b,c in only one place and re-run the program.
def f0(x,y,args) :
    a,b,c = args
    xp = (y-b)/a
    yp = x*x+c
    return xp,yp 

def f1(x,y,args) :
    a,b,c = args
    xp = numpy.sqrt(y-c)
    yp = a*x+b
    return xp,yp 

# --- Here is the relaxation loop.
for i in range(100):
    xp,yp = f1(x,y,constants)
    if (numpy.isclose(x,xp,atol=1.E-6,rtol=0.)) and (numpy.isclose(y,yp,atol=1.E-6,rtol=0.)) :
        print("X : {0:9.5f} Y : {1:8.5f}".format(x,y))
        break
    else :
        x = xp
        y = yp
        print("X : {0:9.5f} Y : {1:8.5f}".format(x,y))

X :   1.04881 Y :  2.90000
X :   1.37840 Y :  2.04881
X :   1.02411 Y :  2.37840
X :   1.17405 Y :  2.02411
X :   1.01199 Y :  2.17405
X :   1.08354 Y :  2.01199
X :   1.00597 Y :  2.08354
X :   1.04093 Y :  2.00597
X :   1.00298 Y :  2.04093
X :   1.02026 Y :  2.00298
X :   1.00149 Y :  2.02026
X :   1.01008 Y :  2.00149
X :   1.00074 Y :  2.01008
X :   1.00503 Y :  2.00074
X :   1.00037 Y :  2.00503
X :   1.00251 Y :  2.00037
X :   1.00019 Y :  2.00251
X :   1.00125 Y :  2.00019
X :   1.00009 Y :  2.00125
X :   1.00063 Y :  2.00009
X :   1.00005 Y :  2.00063
X :   1.00031 Y :  2.00005
X :   1.00002 Y :  2.00031
X :   1.00016 Y :  2.00002
X :   1.00001 Y :  2.00016
X :   1.00008 Y :  2.00001
X :   1.00001 Y :  2.00008
X :   1.00004 Y :  2.00001
X :   1.00000 Y :  2.00004
X :   1.00002 Y :  2.00000
X :   1.00000 Y :  2.00002
X :   1.00001 Y :  2.00000
X :   1.00000 Y :  2.00001
X :   1.00000 Y :  2.00000
X :   1.00000 Y :  2.00000
X :   1.00000 Y :  2.00000
X :   1.00000 Y :  2.00000
X

#### Newton's method.

Newtown's method can be extended in multiple ways to solve for simultaneous nonlinear equations.
$$
\begin{array}{c}
f_1(x_1,\dots,x_N) &=& 0, \\
 \vdots & \\
f_N(x_1,\dots,x_N) &=& 0
\end{array}
$$

We take a Taylor expansion of *each function*, around *each varaible's* inital guess point 
$\left\{x^*_1,\dots,x^*_N\right\}$
$$
f_i(x_1,\dots,x_N) = f_i(x^*_1,\dots,x^*_N)+\sum_j(x_j-x^*_j)\frac{\partial f_i}{\partial x_j}\biggr\rvert_{x^*}+\dots
$$

This is a matrix equation
$$
\vec{f}(\vec{x})=\vec{f}(\vec{x}^*)+\pmb{J}\cdot(\vec{x}-\vec{x}^*)+\dots
$$
where $\pmb{J}$ is the Jacobian matrix $J_{ij}=\partial f_i/\partial x_j|_{x^*}$

As we approach the root, the left hand side becomes zero, and if we define the step of our guess twards the solution as $\Delta\vec{x}=\vec{x}^*-\vec{x}$ we have
$$
\pmb{J}\cdot\Delta\vec{x}=\vec{f}
$$ 
and we can solve for $\Delta\vec{x}$ using linear algebra. A sinlge step in Newton's method is to move the inital guess to a new point
$$
\vec{x}'=\vec{x}^*-\Delta\vec{x}
$$

#### Example 2
Consider the following circuit
<img src="./images/nlcircuit.png" width=200/>
The resistors obey Ohm's law, but the diode obeys the <a href="https://en.wikipedia.org/wiki/Shockley_diode_equation">diode equation</a>
$$I = I_0\left(e^{V/V_T}-1\right)$$
where $V$ is the voltage across the diode, and $I_0$ and $V_T$ are constants. Applying the Kirchoff current law we have at points $V_1$ and $V_2$,
$$f_1 = \frac{V_1+V_+}{R_1} + \frac{V_1-0}{R_2} + I_0\left(e^{(V_1-V_2)/V_T}-1\right)=0$$
$$f_2 = \frac{V_2+V_+}{R_3} + \frac{V_2-0}{R_4} - I_0\left(e^{(V_1-V_2)/V_T}-1\right)=0$$
The first term of the Jacobian is
$$\frac{\partial f_1}{\partial V_1} = \frac{1}{R_1} + \frac{1}{R_2} + \frac{I_0}{V_T}e^{(V_1-V_2)/V_T}$$
I leave it to the reader to calculate the remaining 3 terms.

Use Newton's method to fing $V_1$ and $V_2$ given
$$\begin{array}{c}
V_+ &=& 5\textrm{ V} & V_T &=& 0.05\textrm{ V}\\
R_1 &=& 1\textrm{ k$\Omega$} & R_2 &=& 4\textrm{ k$\Omega$} & R_3 &=& 3\textrm{ k$\Omega$} & R_4 &=& 2\textrm{ k$\Omega$}\\
I_0 &=& 3\textrm{ nA}
\end{array}$$

In [3]:
constants = (   5.0,    # V+ (V)
             1000.0,    # R1 (ohm)
             4000.0,    # R2 (ohm)
             3000.0,    # R3 (ohm)
             2000.0,    # R4 (ohm)
                3.0E-9, # I0 (A)
                0.05)   # VT (V)

# -- In this example, I am packing the variables I want to calculate into an array.
# -- I find this logical, as is should be a vector of solutions.
# -- This is, of course, the initial guess.
v12 = numpy.array([0.1,  # V1
                   0.1]) # V2

# -------------------------------------------------------------
def voltage_function(v12,args) :
    
    # -- I am unpacking the varaibles into new varaibles with the same names as the written problem.
    # -- I find this makes it easier to enter the equation correctly.
    vp,r1,r2,r3,r4,i0,vt = args
    v1,v2 = v12
    
    temp = i0*(numpy.exp((v1-v2)/vt)-1.0)
    f0 = (v1-vp)/r1 + v1/r2 + temp
    f1 = (v2-vp)/r3 + v2/r4 - temp
    
    # -- Since the variables come in as an array, that is how I will return them.
    return numpy.array([f0,f1])
# -------------------------------------------------------------


# -------------------------------------------------------------
def voltage_jacobian(v12,args) :

    vp,r1,r2,r3,r4,i0,vt = args
    v1,v2 = v12
    
    # -- This Jacobian is a 2x2 matrix (see the return statement below)
    # -- Watch out here, indexing by zero is not how we wrote the problem above.
    temp = i0*(numpy.exp((v1-v2)/vt))/vt
    J00 = 1./r1 + 1./r2 + temp
    J01 = -temp
    J10 = -temp
    J11 = 1./r3 + 1./r4 + temp
    
    return numpy.array([[J00,J01],[J10,J11]])
# -------------------------------------------------------------


for i in range(100):
    # -- In each step, we need to find delta-V from the function f and J
    # -- Having them in a vector/matrix format works great for numpy.linalg.solve.
    f = voltage_function(v12, constants)
    J = voltage_jacobian(v12, constants)
    dv = numpy.linalg.solve(J,f)
    # -- Then we step towards the solutions
    v12p = v12 - dv
    if numpy.allclose(v12p, v12, atol=1.E-8,rtol=0.):
        print("X : {0:9.5g} V     Y : {1:8.5g} V".format(v12[0],v12[1]))
        break
    else :
        v12 = v12p
        print("X : {0:9.5g} V     Y : {1:8.5g} V".format(v12[0],v12[1]))

X :    3.9999 V     Y :   2.0001 V
X :    3.9799 V     Y :   2.0301 V
X :    3.9599 V     Y :   2.0601 V
X :    3.9399 V     Y :   2.0901 V
X :    3.9199 V     Y :   2.1201 V
X :    3.8999 V     Y :   2.1501 V
X :    3.8799 V     Y :   2.1801 V
X :    3.8599 V     Y :   2.2101 V
X :    3.8399 V     Y :   2.2401 V
X :    3.8199 V     Y :   2.2701 V
X :    3.7999 V     Y :   2.3001 V
X :    3.7799 V     Y :   2.3301 V
X :    3.7599 V     Y :   2.3601 V
X :    3.7399 V     Y :   2.3901 V
X :    3.7199 V     Y :   2.4201 V
X :    3.6999 V     Y :   2.4501 V
X :    3.6799 V     Y :   2.4801 V
X :    3.6599 V     Y :   2.5101 V
X :    3.6399 V     Y :   2.5401 V
X :    3.6199 V     Y :   2.5701 V
X :    3.5999 V     Y :   2.6001 V
X :    3.5799 V     Y :   2.6301 V
X :    3.5599 V     Y :   2.6601 V
X :      3.54 V     Y :     2.69 V
X :    3.5202 V     Y :   2.7198 V
X :    3.5006 V     Y :   2.7491 V
X :    3.4819 V     Y :   2.7771 V
X :    3.4653 V     Y :   2.8021 V
X :    3.4532 V     

#### Example 2 continued
In <a href="https://docs.scipy.org/doc/scipy/reference/optimize.html">`scipy.optimize`</a> there are an amazing array of routines for solving nonlinear equations an root finding. This problem is <a href="https://docs.scipy.org/doc/scipy/reference/optimize.html#multidimensional">multidimensional root finding</a>, so we should use <a href="https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.root.html#scipy.optimize.root">`root`</a>. There are methods that use the Jacobian, and many that do not. Use one of each kind to verify the answer above.

In [13]:
constants = (   5.0,    # V+ (V)
             1000.0,    # R1 (ohm)
             4000.0,    # R2 (ohm)
             3000.0,    # R3 (ohm)
             2000.0,    # R4 (ohm)
                3.0E-9, # I0 (A)
                0.05)   # VT (V)

# -------------------------------------------------------------
def voltage_function(v12,*args) :
    
    vp,r1,r2,r3,r4,i0,vt = args
    v1,v2 = v12
    
    temp = i0*(numpy.exp((v1-v2)/vt)-1.0)
    f0 = (v1-vp)/r1 + v1/r2 + temp
    f1 = (v2-vp)/r3 + v2/r4 - temp
    
    return numpy.array([f0,f1])
# -------------------------------------------------------------


# -------------------------------------------------------------
def voltage_jacobian(v12,*args) :

    vp,r1,r2,r3,r4,i0,vt = args
    v1,v2 = v12
    
    temp = i0*(numpy.exp((v1-v2)/vt))/vt
    J00 = 1./r1 + 1./r2 + temp
    J01 = -temp
    J10 = -temp
    J11 = 1./r3 + 1./r4 + temp
    
    return numpy.array([[J00,J01],[J10,J11]])
# -------------------------------------------------------------


# -- The first method we will use is the non-Jacobian method 'hybr'
# -- Find the roots of a multivariate function using MINPACK’s hybrd and hybrj routines (modified Powell method).
# -- https://en.wikipedia.org/wiki/MINPACK
v12 = numpy.array((3.0,  # V1
                   2.0)) # V2
res = scipy.optimize.root(voltage_function,v12,args=constants,method='hybr')
print("HYBR method message : ",res.message)
print(res.x)
# -- The second method we will use is the method 'lm' which needs the jacobian
# -- Solve for least squares with Levenberg-Marquardt
# -- https://en.wikipedia.org/wiki/Levenberg%E2%80%93Marquardt_algorithm
v12 = numpy.array((3.0,  # V1
                   3.0)) # V2
res = scipy.optimize.root(voltage_function,v12,args=constants,jac=voltage_jacobian,method='lm')
# print(res.keys())
print("LM method message : ",res.message)
print(res.x)

HYBR method message :  The solution converged.
[3.44695462 2.82956807]
dict_keys(['x', 'message', 'status', 'success', 'cov_x', 'fun', 'nfev', 'njev', 'fjac', 'ipvt', 'qtf'])
LM method message :  The relative error between two consecutive iterates is at most 0.000000
[3.44695462 2.82956807]


#### Optimization
Mathematical optimization deals with the problem of numerically finding minimums (or maximums, or zeros) of a function. In this context, the function we wish to find the maximum of is called *cost function*, or sometimes the *objective function*, or in some cases the *energy*.

Here, we are interested in using [`scipy.optimize`][1] as a black-box. We are not going into the details of how the different methods work, but we should still learn to appriciate their differences because not all optimization problems are equal. Knowing your problem enables you to choose the right tool. Sometimes having a Jacobian will really help as well. Note that Newton's method is pretty much the starting point for many of these methods.

Read through this page on [optimization][2] for examples.

Let's work through some [`scipy.optimize.minimize`][3] examples and excercises in 1 and 2 dimensions.

[1]: https://docs.scipy.org/doc/scipy/reference/optimize.html
[2]: http://scipy-lectures.org/advanced/mathematical_optimization/
[3]: https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html

#### Example 3    1-D

Find the minimum of the equation $f(x)=e^{(x-a)^2}$ for $a=0.7$. Use the Jacobian.

In [5]:
constant = (0.7,)

def f(x,*args) :
    return numpy.exp((x[0]-args[0])**2)

def J(x, *args) :
    t = (x[0]-args[0])
    return 2*t*numpy.exp(t**2)

guess = 2.5
res = scipy.optimize.minimize(f,guess,args=constant,method='Newton-CG',jac=J)
print(res)

     fun: 1.0
     jac: array([-7.83642928e-10])
 message: 'Optimization terminated successfully.'
    nfev: 9
    nhev: 0
     nit: 9
    njev: 17
  status: 0
 success: True
       x: array([0.7])


#### Example 4    2D

Find the minimum of the function
$$
f(x,y) = \sqrt{(x-a)^2+(y-b)^2}
$$ 
for $a=2$, $b=20$.
This is actually a difficult problem, because the Jacobian is poorly behaved near the solution. Solving the same problem for $f^2(x)$ is the same problem with the same solutions. Changing the problem like this is called pre-conditioning.

Try finding one solution with constraints on the solutions, keeping them within the bounds of $a=[-3,3]$, and $b=[1.5,12.5]$.

In [6]:
constant = (2.0,20.0)

def f(x,*args) :
    a,b = args
    temp = numpy.sqrt(((x[0]-a)**2)+((x[1]-b)**2))
    return temp

def J(x, *args) :
    a,b = args
    temp = numpy.sqrt(((x[0]-a)**2)+((x[1]-b)**2))
    Ja = (x[0]-a)/temp
    Jb = (x[1]-b)/temp
    return numpy.array([Ja,Jb])
 
guess = numpy.array([1.1,15.5])

res = scipy.optimize.minimize(f,guess,args=constant,method="BFGS",jac=J)
print(res,"\n")
res = scipy.optimize.minimize(f,guess,args=constant,method="BFGS")
print(res,"\n")
res = scipy.optimize.minimize(f,guess,args=constant,method="Nelder-Mead")
print(res,"\n")

      fun: 4.589117562233506
 hess_inv: array([[1, 0],
       [0, 1]])
      jac: array([-0.19611614, -0.98058068])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 53
      nit: 0
     njev: 48
   status: 2
  success: False
        x: array([ 1.1, 15.5]) 

      fun: 1.9567661551385958e-09
 hess_inv: array([[1.53093699e-08, 4.16352467e-10],
       [4.16352467e-10, 4.81279046e-09]])
      jac: array([0.82943351, 0.74736886])
  message: 'Desired error not necessarily achieved due to precision loss.'
     nfev: 254
      nit: 7
     njev: 80
   status: 2
  success: False
        x: array([ 2., 20.]) 

 final_simplex: (array([[ 2.00003293, 19.99998227],
       [ 1.99996508, 20.00003989],
       [ 2.00002588, 19.99995081]]), array([3.74029573e-05, 5.30167465e-05, 5.55816145e-05]))
           fun: 3.740295732252786e-05
       message: 'Optimization terminated successfully.'
          nfev: 85
           nit: 44
        status: 0
       success: True
    

In [7]:
constant = (2.0,20.0)

def f(x,*args) :
    a,b = args
    temp = numpy.sqrt(((x[0]-a)**2)+((x[1]-b)**2))
    return temp

def J(x, *args) :
    a,b = args
    temp = numpy.sqrt(((x[0]-a)**2)+((x[1]-b)**2))
    Ja = (x[0]-a)/temp
    Jb = (x[1]-b)/temp
    return numpy.array([Ja,Jb])
 
guess = numpy.array([1.1,15.5])

res = scipy.optimize.minimize(f,guess,args=constant,method="POWELL")
print(res,"\n")
res = scipy.optimize.minimize(f,guess,args=constant,method="CG",jac=J)
print(res,"\n")
res = scipy.optimize.minimize(f,guess,args=constant,method="SLSQP",bounds=((-3., 3.), (1.5, 12.5)))
print(res,"\n")

   direc: array([[-6.40841120e-07, -1.26871144e-05],
       [ 3.91337640e-13, -3.47732042e-15]])
     fun: 0.0
 message: 'Optimization terminated successfully.'
    nfev: 358
     nit: 7
  status: 0
 success: True
       x: array([ 2., 20.]) 

     fun: 4.589117562233506
     jac: array([-0.19611614, -0.98058068])
 message: 'Desired error not necessarily achieved due to precision loss.'
    nfev: 53
     nit: 0
    njev: 48
  status: 2
 success: False
       x: array([ 1.1, 15.5]) 

     fun: 7.500000241639889
     jac: array([ 2.53856182e-04, -9.99999940e-01])
 message: 'Optimization terminated successfully'
    nfev: 12
     nit: 4
    njev: 4
  status: 0
 success: True
       x: array([ 2.00190384, 12.5       ]) 



In [8]:
constant = (2.0,20.0)

def f(x,*args) :
    a,b = args
    temp = ((x[0]-a)**2)+((x[1]-b)**2)
    return temp

def J(x, *args) :
    a,b = args
    temp = ((x[0]-a)**2)+((x[1]-b)**2)
    Ja = 2.0*(x[0]-a)
    Jb = 2.0*(x[1]-b)
    return numpy.array([Ja,Jb])
 
guess = numpy.array([1.1,15.5])

res = scipy.optimize.minimize(f,guess,args=constant,method="BFGS",jac=J)
print(res,"\n")
res = scipy.optimize.minimize(f,guess,args=constant,method="BFGS")
print(res,"\n")
res = scipy.optimize.minimize(f,guess,args=constant,method="Nelder-Mead")
print(res,"\n")

      fun: 1.9721522630525295e-31
 hess_inv: array([[ 0.98076923, -0.09615385],
       [-0.09615385,  0.51923077]])
      jac: array([8.8817842e-16, 0.0000000e+00])
  message: 'Optimization terminated successfully.'
     nfev: 4
      nit: 3
     njev: 4
   status: 0
  success: True
        x: array([ 2., 20.]) 

      fun: 8.19105207292833e-14
 hess_inv: array([[ 0.98076921, -0.09615389],
       [-0.09615389,  0.51923078]])
      jac: array([-5.53844051e-07,  7.94841810e-08])
  message: 'Optimization terminated successfully.'
     nfev: 12
      nit: 3
     njev: 4
   status: 0
  success: True
        x: array([ 1.99999972, 20.00000003]) 

 final_simplex: (array([[ 2.00003293, 19.99998227],
       [ 1.99996508, 20.00003989],
       [ 2.00002588, 19.99995081]]), array([1.39898122e-09, 2.81077541e-09, 3.08931587e-09]))
           fun: 1.3989812164708407e-09
       message: 'Optimization terminated successfully.'
          nfev: 85
           nit: 44
        status: 0
       success: Tru